import asyncio
import json
import os
import logging
import requests
import ssl
import time
from urllib.parse import urlencode
import websockets

from datetime import datetime
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from binus_rest import *
import inputs
from utils import convert_ms_to_dt, read_from_pickle, write_to_pickle, log


## TO-DO
# bug (fills 2x on first try)
# bug (quantity skews incorrectly?)
# bug (quotes widen out after reversion fills?)        pending



# Logging
logging.basicConfig(filename='main.log', encoding='utf-8', level=logging.DEBUG,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')

logging.getLogger("urllib3").setLevel(logging.WARNING)
logging.getLogger("websockets").setLevel(logging.WARNING)


def init_globals(global_var_path):

    ts = int(time.time() * 1000)

    if os.path.exists(global_var_path):

        global_vars = read_from_pickle(global_var_path)

        if 'HEARTBEAT_TS' in global_vars:

            # if ts lt last heartbeat + HEARTBEAT_BUFFER, reload global_vars
            # (ie if we get disconnceted, remember position)
            if ts < global_vars['HEARTBEAT_TS'] + inputs.HEARTBEAT_TS_BUFFER:
                logging.debug(f'RE-LOADING {global_var_path}')
                return global_vars
            # else:

        logging.debug(f'RE-WRITING {global_var_path}')
        os.remove(global_var_path)



    # if os.path.exists(global_var_path):
    #     logging.info(f're-writing {global_var_path}')
    #     os.remove(global_var_path)

    global_vars = {
        # Define start and stop of CYCLE
        'ACTIVE_CYCLE': False,
        'INIT_BIDS': [],
        'INIT_ASKS': [],
        'STOP_LOSS': [],
        'VOL_ADJUST_MULT': 1, 
        'TIMESTAMP_START_MS_CYCLE': ts,
        # LAST_BID_FILL_TS = 
        # LAST_ASK_FILL_TS = 
        # Total
        'AVG_BUY_PX': 0,
        'AVG_SELL_PX': 0,
        'TIMESTAMP_START_MS': ts,
        'TIMESTAMP_STOP_LOSS_FILL': 0, # init first STOP_LOSS_FILL to value in past
        # 
        'HEARTBEAT_TS': ts, # this is last timestamp
    }
    write_to_pickle(global_vars, global_var_path)

    return global_vars

global_vars = init_globals(inputs.GLOBAL_VAR_PATH)

def reset_cycle_modifier(is_stop_loss):
    logging.debug('RESETTING CYCLE')
    ts = int(time.time() * 1000)
    global_vars['ACTIVE_CYCLE'] = False
    global_vars['STOP_LOSS'] = []
    global_vars['TIMESTAMP_START_MS_CYCLE'] = ts
    if is_stop_loss:
        global_vars['TIMESTAMP_STOP_LOSS_FILL'] = ts
    else:
        global_vars['TIMESTAMP_STOP_LOSS_FILL'] = 0
    write_to_pickle(global_vars, inputs.GLOBAL_VAR_PATH)


def get_avg_buys_and_sells(trades_history, ts_start, lookback_num=None):
    """
    input ... trades = get_trade_history_pretty(inputs.SYMBOL)
          ... ts_start = global_vars['TIMESTAMP_START_MS']
    """
    trades = trades_history[trades_history.time >= ts_start]
    if len(trades) > 0:
        last_px = float(trades['price'].iloc[-1])

        buys = trades[trades['isBuyer'] == True]
        sells = trades[trades['isBuyer'] == False]

        if lookback_num:
            buys = buys[buys.orderId.isin(buys.orderId.unique()[-lookback_num:])].copy()
            sells = sells[sells.orderId.isin(sells.orderId.unique()[-lookback_num:])].copy()
        
        if not buys.empty:
            avg_buy_px = buys['quoteQty'].astype(float).sum() / buys['qty'].astype(float).sum()
            last_buy = float(buys['price'].iloc[-1])
            buy_qty = buys['qty'].astype(float).sum()
        else:
            avg_buy_px = np.nan
            last_buy = np.nan
            buy_qty = 0

        if not sells.empty:
            avg_sell_px = sells['quoteQty'].astype(float).sum() / sells['qty'].astype(float).sum()
            last_sell = float(sells['price'].iloc[-1])
            sell_qty = sells['qty'].astype(float).sum()
        else:
            avg_sell_px = np.nan
            last_sell = np.nan
            sell_qty = 0

        net_qty = buy_qty - sell_qty

        return {
            'TIME_START': datetime.fromtimestamp(ts_start / 1000).strftime('%d%b%y %H:%m:%S').upper(),
            'AVG_BUY_PX': avg_buy_px,
            'AVG_SELL_PX': avg_sell_px,
            'NET_QTY': net_qty,
            'NET_QTY_USD': net_qty * last_px,
            'TOTAL_QTY': buy_qty + sell_qty,
            'TOTAL_QTY_USD': last_px * (buy_qty + sell_qty),
            'LAST_BUY': last_buy,
            'LAST_SELL': last_sell,
        }

    return {
        'TIME_START': datetime.fromtimestamp(ts_start / 1000).strftime('%d%b%y %H:%m:%S').upper(),
        'AVG_BUY_PX': np.nan,
        'AVG_SELL_PX': np.nan,
        'NET_QTY': 0,
        'NET_QTY_USD': 0,
        'TOTAL_QTY': 0,
        'TOTAL_QTY_USD': 0,
        'LAST_BUY': np.nan,
        'LAST_SELL': np.nan,
    }


def pretty_quote_logging(open_bid, open_ask, key):

    # logging.info('\n\n')
    logging.info(f'************ CURRENT QUOTES {key}:')
    for n, i in enumerate(open_ask[::-1], 1):
        logging.info(f'Ask {len(open_ask) + 1 - n}: ({i[1]}, {i[2]})')
    logging.info('-'*20)
    for n, i in enumerate(open_bid, 1):
        logging.info(f'Bid {n}: ({i[1]}, {i[2]})')
    # logging.info('\n\n')


def pretty_bid_and_ask(opens):
    open_bid = [(d['orderId'], float(d['price']), float(d['origQty'])) for d in opens if (d['side'] == 'BUY' and d['type'] == 'LIMIT')]
    open_ask = [(d['orderId'], float(d['price']), float(d['origQty'])) for d in opens if (d['side'] == 'SELL' and d['type'] == 'LIMIT')]
    # sort so best is at index 0
    open_bid.sort(key=lambda x: x[1], reverse=True)
    open_ask.sort(key=lambda x: x[1])
    return open_bid, open_ask


def get_best_order(orders_raw, tgt_qty):
    orders = [[float(i[0]), float(i[1])] for i in orders_raw]
    cum_qty = 0
    proceeds = 0
    for px, qty in orders:
        cum_qty += qty
        proceeds += px * qty
        if cum_qty >= tgt_qty:
            break
    best_px = proceeds / cum_qty if cum_qty > 0 else np.nan
    return best_px


def get_reversion(klines):

    df = convert_klines(klines)
    hi_lo = pd.concat([df['High'], df['Low']]).sort_index()
    hi_lo = hi_lo.astype(float)
    hi_lo.index = range(len(hi_lo))

    rolling_std = hi_lo.rolling(inputs.BOLL_LOOKBACK_INTERVAL_LT, min_periods=1).std()
    rolling_ma = hi_lo.rolling(inputs.BOLL_LOOKBACK_INTERVAL_LT, min_periods=1).mean() 
    bolu = rolling_ma + rolling_std * 2   
    bold = rolling_ma - rolling_std * 2

    ## S/T graph
    bolu_loc = bolu.iloc[-inputs.BOLL_LOOKBACK_INTERVAL_LT:]
    hilo_loc = hi_lo.iloc[-inputs.BOLL_LOOKBACK_INTERVAL_LT:]
    bold_loc = bold.iloc[-inputs.BOLL_LOOKBACK_INTERVAL_LT:]

    # if price > upp bollinger band OR price < lower bolliger band
    # for consecutive BOLL_CROSS_INTERVAL_HOURLY, then dont quote (we are in "momentum")
    hilo_gt_bolu = (hilo_loc >= bolu_loc).iloc[-inputs.BOLL_NUM_INTERVAL_IS_MOMENTUM:]
    hilo_lt_bold = (hilo_loc <= bold_loc).iloc[-inputs.BOLL_NUM_INTERVAL_IS_MOMENTUM:]
    in_momentum = hilo_gt_bolu.all() or hilo_lt_bold.all()
    in_reversion = True ##not(in_momentum)
    logging.info(f'in_reversion: {in_reversion}')

    # VOLATILTY-ADJUSTED QUOTING
    lt_vol_ann = inputs.VOL_BASELINE_ANNUALIZED
    st_vol = hi_lo.iloc[-inputs.BOLL_LOOKBACK_INTERVAL_LT:].pct_change().std()
    st_vol_ann = (st_vol * np.sqrt(4 * 24 * 365.25)) * 100 # need map from 15m klines --> 
    vol_adj_mult = min(max(1, st_vol_ann / lt_vol_ann), inputs.VOL_QUOTING_MAX_ADJUST)
    logging.info(f'lt_vol_ann: {round(lt_vol_ann, 2)}, st_vol_ann: {round(st_vol_ann, 2)}, vol_adj_mult: {round(vol_adj_mult, 3)}')

    return in_reversion, vol_adj_mult



if __name__ == "__main__":

    def main(msg, num):

        logging.info(f'START_TIME: {datetime.now().strftime("%Y-%m-%d %H-%M-%S")}')

        ts_now = int(time.time() * 1000)
        if ts_now < global_vars['TIMESTAMP_STOP_LOSS_FILL'] + inputs.STOP_LOSS_FILL_BUFFER_MS:
            logging.debug('IN STOP_LOSS_FILL DELAY PERIOD')
            return

        ############################################################### WSS DATA
        # How to add multiple streams and cache all the results?
        # ie /orders , /trades , /depth

        j = json.loads(msg)

        # bids + offers
        if ('e' in j) and (j['e'] == 'depthUpdate'): 
            bids_tmp = j['b']
            asks_tmp = j['a']
            best_bid_px = get_best_order(bids_tmp, .01) # <<<<<<<<<<<<<<<<<<<< HARD-CODED 
            best_ask_px = get_best_order(asks_tmp, .01) # <<<<<<<<<<<<<<<<<<<< HARD-CODED 
            mid_px = (best_bid_px + best_ask_px) / 2.
            print(f'wss stream: {round(best_bid_px, 2)}, {round(mid_px, 2)}, {round(best_ask_px, 2)}')
            if (best_bid_px != best_bid_px) or (best_ask_px != best_ask_px):
                logging.debug(f'skip iteration, nans detected in price')    
                return
        else:
            return


        ############################################################### INIT ORDERS
        # logging.info(f'num: {num}')
        if num % 15 == 0:

            klines = get_klines_data(inputs.SYMBOL, limit=250, interval=inputs.BOLL_LOOKBACK_KLINES_INTERVAL)            
            in_reversion, vol_adj_mult = get_reversion(klines)
            global_vars['VOL_ADJUST_MULT'] = vol_adj_mult # call every 15 iterations to save on rate_limiting

            no_open_orders = not(any(get_open_orders(inputs.SYMBOL))) ## TO / DO ------------> RENAME get_all_open_orders()
            logging.info(f'in_reversion: {in_reversion}')    
            logging.info(f'no_open_orders: {no_open_orders}')    
            init_orders = in_reversion & no_open_orders

            if init_orders:

                ############################################################### DEFINE INIT ORDERS

                order_base_sz = inputs.MAX_USD_RISK / mid_px / inputs.NUM_ORDERS
                base_qty = round(order_base_sz, PRECISION_DECIMALS[inputs.BASE])

                logging.info(f'vol_adjusted ORDER_INTERVAL_PCT: {inputs.ORDER_INTERVAL_PCT} --> {inputs.ORDER_INTERVAL_PCT * vol_adj_mult}')

                # bid_best = mid_px - (mid_px * inputs.MID_INTERVAL_PCT * vol_adj_mult / 2.)
                # ask_best = mid_px + (mid_px * inputs.MID_INTERVAL_PCT * vol_adj_mult / 2.)

                bid_best = mid_px * (1 - inputs.MID_INTERVAL_PCT * vol_adj_mult)
                ask_best = mid_px * (1 + inputs.MID_INTERVAL_PCT * vol_adj_mult)

                bids = np.full(inputs.NUM_ORDERS, np.nan)
                asks = np.full(inputs.NUM_ORDERS, np.nan)
                bids[0] = bid_best
                asks[0] = ask_best

                for n in range(1, inputs.NUM_ORDERS):
                    # bids[n] = bids[n - 1] - bids[n - 1] * inputs.ORDER_INTERVAL_PCT * vol_adj_mult
                    # asks[n] = asks[n - 1] + asks[n - 1] * inputs.ORDER_INTERVAL_PCT * vol_adj_mult
                    bids[n] = bids[n - 1] * (1 - (inputs.ORDER_INTERVAL_PCT + inputs.ORDER_INTERVAL_INCR_PCT * n) * vol_adj_mult)
                    asks[n] = asks[n - 1] * (1 + (inputs.ORDER_INTERVAL_PCT + inputs.ORDER_INTERVAL_INCR_PCT * n) * vol_adj_mult)


                assert ask_best > mid_px, f'best ask {ask_best} cannot be < mid {mid_px}'
                assert bid_best < mid_px, f'best bid {bid_best} cannot be > mid {mid_px}'

                logging.info('******** INIT ORDERS ********')
                logging.info(f'bids: {bids}')
                logging.info(f'best bid px: {best_bid_px}')
                logging.info(f'mid_px: {mid_px}')
                logging.info(f'best ask px: {best_ask_px}')
                logging.info(f'asks: {asks}')

                ############################################################### PLACE INIT ORDERS
                
                for n, bid in enumerate(bids, 1):
                    r = post_order(
                            base=inputs.BASE,
                            quote=inputs.QUOTE,
                            side='buy',
                            type='limit',
                            quantity=base_qty, 
                            price=bid
                        )
                    logging.info(r)
                    ## time.sleep(.2)

                for n, ask in enumerate(asks, 1):
                    r = post_order(
                            base=inputs.BASE,
                            quote=inputs.QUOTE,
                            side='sell',
                            type='limit',
                            quantity=base_qty, 
                            price=ask
                        )
                    logging.info(r)
                    ## time.sleep(.2)

                # start of cycle
                global_vars['ACTIVE_CYCLE'] = True
                global_vars['INIT_BIDS'] = bids
                global_vars['INIT_ASKS'] = asks
                write_to_pickle(global_vars, inputs.GLOBAL_VAR_PATH)



        ############################################################### ADD VOL ADJUST MULT
        ###############################################################

        vol_adjust_mult = global_vars['VOL_ADJUST_MULT']


        ############################################################### CHECK OPEN ORDERS
        ###############################################################

        # Get LIMIT orders
        # Note: OCO orders show up as combo of STOP_LOSS_LIMIT and LIMIT MAKER , and they share same orderIdList
        opens = get_open_orders(inputs.SYMBOL)
        if 'code' in opens:
            raise ValueError(opens)
        open_bid, open_ask = pretty_bid_and_ask(opens)
        pretty_quote_logging(open_bid, open_ask, key='start')


        ############################################################### GET FILLS
        ###############################################################

        trades_history = get_trade_history_pretty(inputs.SYMBOL) # GET/ orders
        # avg prices from cycle start
        avgs_cycle = get_avg_buys_and_sells(trades_history, global_vars['TIMESTAMP_START_MS_CYCLE'], lookback_num=inputs.NUM_ORDERS)
        # avg prices from run start
        avgs = get_avg_buys_and_sells(trades_history, global_vars['TIMESTAMP_START_MS'])

        ############################################################### CHECK CYCLE
        ###############################################################

        # if either of the STOP_LOSS was filled, end cycle
        opens_oid = [d['orderId'] for d in opens]
        exit_filled = [oid for oid in global_vars['STOP_LOSS'] if oid not in opens_oid]

        if any(exit_filled):
            # condition if STOP_LOSS is the leg that filled
            sl_filled = not any([d['type'] == 'STOP_LOSS_LIMIT' for d in opens])
            # delete orders and reset cycle
            delete_open_order(inputs.SYMBOL)
            reset_cycle_modifier(is_stop_loss=sl_filled)

        ############################################################### ORDER UPDATING LOGIC
        ###############################################################

        if global_vars['ACTIVE_CYCLE']:

            base_risk = avgs['NET_QTY']
            max_base_risk = inputs.MAX_USD_RISK / mid_px
            avg_buy = avgs_cycle['AVG_BUY_PX']
            avg_sell = avgs_cycle['AVG_SELL_PX']
            last_buy = avgs_cycle['LAST_BUY']
            last_sell = avgs_cycle['LAST_SELL']
            init_bids = global_vars['INIT_BIDS']
            init_asks = global_vars['INIT_ASKS']

            if not(global_vars['STOP_LOSS']):


                if any(open_bid) and any(open_ask): ## and (num > 0):

                    # >>>>>>>>>>>> 1) Quote aggressiveness (BPS)
                    # how much more aggressive to quote to get out of position
    
                    # INVENTORY SKEW
                    # 1 = max long position
                    # -1 = max short position
                    # 0 = neutral
                    inv_skew = base_risk / max_base_risk
                    # skew prices (relative to best on other side)
                    # bid_skew_pct = inv_skew * inputs.MAX_INV_SKEW_PCT ## min(0, inv_skew)
                    # ask_skew_pct = inv_skew * -1. * inputs.MAX_INV_SKEW_PCT ## max(0, inv_skew)
                    logging.debug(f'inv_skew: {inv_skew}')

                    logging.debug(f'restacking bids (start): {open_bid}')
                    logging.debug(f'restacking asks (start): {open_ask}')

                    # DEFINE BEST_BID + BEST_ASK            

                    # If bid gets lifted 
                    if len(open_bid) < inputs.NUM_ORDERS:
                        # best_bid = last_buy * (1 - inputs.ORDER_INTERVAL_PCT)
                        # best_ask = last_buy * (1 + inputs.MIN_REVERSION_BPS_CAPTURE)
                        best_bid = last_buy * (1 - inputs.ORDER_INTERVAL_PCT * vol_adjust_mult) ## avg_buy 
                        best_ask = avg_buy * (1 + inputs.MIN_REVERSION_BPS_CAPTURE * vol_adjust_mult)
                        logging.debug('If bid gets lifted ')

                    # If ask gets lifted
                    elif len(open_ask) < inputs.NUM_ORDERS:
                        # best_bid = last_sell * (1 - inputs.MIN_REVERSION_BPS_CAPTURE)
                        # best_ask = last_sell * (1 + inputs.ORDER_INTERVAL_PCT)
                        best_bid = avg_sell * (1 - inputs.MIN_REVERSION_BPS_CAPTURE * vol_adjust_mult)
                        best_ask = last_sell * (1 + inputs.ORDER_INTERVAL_PCT * vol_adjust_mult) ## avg_sell
                        logging.debug('If ask gets lifted')

                    # If neither , continue as normal
                    else:
                        best_bid = open_bid[0][1]
                        best_ask = open_ask[0][1]
                        logging.debug('neither bid or ask lifted')

                    # DEFINE ARRAYS
                    rsz_bid_px = np.full(inputs.NUM_ORDERS, np.nan)
                    rsz_bid_px[0] = best_bid
                    for n in range(1, inputs.NUM_ORDERS):
                        rsz_bid_px[n] = rsz_bid_px[n - 1] * (1 - (inputs.ORDER_INTERVAL_PCT * vol_adjust_mult + inputs.ORDER_INTERVAL_INCR_PCT * n))

                    rsz_ask_px = np.full(inputs.NUM_ORDERS, np.nan)
                    rsz_ask_px[0] = best_ask
                    for n in range(1, inputs.NUM_ORDERS):
                        rsz_ask_px[n] = rsz_ask_px[n - 1] * (1 + (inputs.ORDER_INTERVAL_PCT * vol_adjust_mult + inputs.ORDER_INTERVAL_INCR_PCT * n))


                    logging.debug(f'(best_bid: {round(best_bid,2)},  best_ask: {round(best_ask, 2)},  last_sell: {round(last_sell, 2)},  last_buy: {round(last_buy, 2)},  mid_px: {round(mid_px,2)})')
                    logging.debug(f'restacking bids (end): {rsz_bid_px}')
                    logging.debug(f'restacking asks (end): {rsz_ask_px}')

                    # >>>>>>>>>>>> 2) Quote aggressiveness (size)
                    # how much size to put into positions
                        
                    # BASE RISK TO QUOTE
                    tot_qty_ask = (max_base_risk + base_risk)
                    qty_ea_ask = tot_qty_ask / inputs.NUM_ORDERS 
                
                    tot_qty_bid = (max_base_risk - base_risk) 
                    qty_ea_bid = tot_qty_bid / inputs.NUM_ORDERS

                    # # DEFINE QTY SKEW
                    # # put more quantity at the front depending on current inventory
                    # qty_skew = max(1, abs(inv_skew) * inputs.MAX_ORDER_WGT_SKEW)
                    
                    # # normalize to tot_qty_ask
                    # lin_ask = np.linspace(
                    #     start=qty_ea_ask * qty_skew, 
                    #     stop=qty_ea_ask / qty_skew, 
                    #     num=inputs.NUM_ORDERS)
                    
                    # rsz_ask_qty = lin_ask / (sum(lin_ask) / tot_qty_ask) # adjust quantity
                    # rsz_ask_qty = rsz_ask_qty.clip(min=inputs.MIN_TRADE_USD / mid_px)

                    # lin_bid = np.linspace(
                    #     start=qty_ea_bid * qty_skew, 
                    #     stop=qty_ea_bid / qty_skew, 
                    #     num=inputs.NUM_ORDERS)
                    
                    # rsz_bid_qty = lin_bid / (sum(lin_bid) / tot_qty_bid) # adjust quantity  
                    # rsz_bid_qty = rsz_bid_qty.clip(min=inputs.MIN_TRADE_USD / mid_px)


                    qty_ea_ask = max(qty_ea_ask, inputs.MIN_TRADE_USD / mid_px)
                    qty_ea_bid = max(qty_ea_bid, inputs.MIN_TRADE_USD / mid_px)


                    # logging.debug(f'rsz_bid_qty: {rsz_bid_qty}')
                    # logging.debug(f'rsz_ask_qty: {rsz_ask_qty}')

                    # UPDATE ORDERS

                    ## if not np.isnan(rsz_ask_px).all():
                    tmp = 0
                    # for a_px_new, a_qty_new in zip(rsz_ask_px, rsz_ask_qty):
                    for a_px_new in rsz_ask_px:

                        ## apply skew
                        # a_px_new = a_px_new * (1 + ask_skew_pct)
                        # logging.debug(f'{num} asked skewed (end): {a_px_new}')

                        try:
                            oid, a_px_old, a_qty_old = open_ask[tmp]
                            ask_px_update_thresh = abs(a_px_new / a_px_old - 1) >= inputs.ORDER_PX_UPDATE_PCT
                            # ask_qty_update_thresh = abs(a_qty_new / a_qty_old - 1) >= inputs.ORDER_QTY_UPDATE_PCT
                            ask_qty_update_thresh = abs(qty_ea_ask / a_qty_old - 1) >= inputs.ORDER_QTY_UPDATE_PCT
                        except IndexError:
                            oid = np.nan
                            ask_px_update_thresh = True
                            ask_qty_update_thresh = True
                            a_px_old = np.nan
                            a_qty_old = np.nan

                        if ask_px_update_thresh or ask_qty_update_thresh:
                            logging.debug(f'changing ask px from {round(a_px_old, 2)} to {round(a_px_new, 2)}')
                             #logging.debug(f'changing ask qty from {round(a_qty_old, 6)} to {round(a_qty_new, 6)}')
                            logging.debug(f'changing ask qty from {round(a_qty_old, 6)} to {round(qty_ea_ask, 6)}')

                            delete_open_by_order_id(inputs.SYMBOL, [oid])
                            rask = post_order(
                                base=inputs.BASE,
                                quote=inputs.QUOTE,
                                side='sell',
                                type='limit',
                                quantity=qty_ea_ask, ##a_qty_new, 
                                price=a_px_new
                            )
                            logging.debug(rask)
                        else:
                            logging.debug(f'New ask {round(a_px_new, 2)} too close to old ask {round(a_px_old, 2)}')
                        tmp += 1

                    ## if not np.isnan(rsz_bid_px).all():
                    tmp = 0
                    # for b_px_new, b_qty_new in zip(rsz_bid_px, rsz_bid_qty):
                    for b_px_new in rsz_bid_px:
                    
                        ## apply skew
                        # b_px_new = b_px_new * (1 - bid_skew_pct)
                        # logging.debug(f'{num} bid skewed (end): {b_px_new}')

                        try:
                            oid, b_px_old, b_qty_old = open_bid[tmp]
                            bid_px_update_thresh = abs(b_px_new / b_px_old - 1) >= inputs.ORDER_PX_UPDATE_PCT
                            # bid_qty_update_thresh = abs(b_qty_new / b_qty_old - 1) >= inputs.ORDER_QTY_UPDATE_PCT
                            bid_qty_update_thresh = abs(qty_ea_bid / b_qty_old - 1) >= inputs.ORDER_QTY_UPDATE_PCT
                        except IndexError:
                            oid = np.nan
                            bid_px_update_thresh = True
                            bid_qty_update_thresh = True
                            b_px_old = np.nan
                            b_qty_old = np.nan
                        
                        if bid_px_update_thresh or bid_qty_update_thresh:
                            logging.debug(f'changing bid px from {round(b_px_old, 2)} to {round(b_px_new, 2)}')
                            # logging.debug(f'changing bid qty from {round(b_qty_old, 6)} to {round(b_qty_new, 6)}')
                            logging.debug(f'changing bid qty from {round(b_qty_old, 6)} to {round(qty_ea_bid, 6)}')

                            delete_open_by_order_id(inputs.SYMBOL, [oid])
                            rbid = post_order(
                                base=inputs.BASE,
                                quote=inputs.QUOTE,
                                side='buy',
                                type='limit',
                                quantity=qty_ea_bid, ##b_qty_new, 
                                price=b_px_new
                            )
                            logging.debug(rbid)
                        else:
                            logging.debug(f'New bid {round(b_px_new, 2)} too close to old bid {round(b_px_old, 2)}')
                        tmp += 1



                elif abs(base_risk) * mid_px > inputs.MAX_USD_RISK / (1 + .01):
                # elif (not(open_ask) or not(open_bid)):

                    if not any(open_bid):
                        logging.debug('bids swept; placing take-profit and stop-loss orders')
                        side = 'sell'
                        # take profit px (last chance for reversion pnl)
                        # take_px = avgs_cycle['AVG_BUY_PX'] * (1 + inputs.TAKE_PROFIT_PCT) 
                        take_px = best_bid_px * (1 + inputs.TAKE_PROFIT_PCT)  ### $$$$$$$$$$$$$$$$$$$$$$$$
                        # stop loss px (cut losses if keeps moving against you)
                        stop_px = last_buy * (1 - inputs.STOP_LOSS_PCT)
                    else:
                        logging.debug('asks swept; placing take-profit and stop-loss orders')
                        side = 'buy'
                        # take profit px (last chance for reversion pnl)
                        # take_px = avgs_cycle['AVG_SELL_PX'] * (1 - inputs.TAKE_PROFIT_PCT)
                        take_px = best_ask_px * (1 - inputs.TAKE_PROFIT_PCT) ### $$$$$$$$$$$$$$$$$$$$$$$$
                        # stop loss px (cut losses if keeps moving against you)
                        stop_px = last_sell * (1 + inputs.STOP_LOSS_PCT)

                    # clear all orders
                    delete_open_order(inputs.SYMBOL)

                    # take-profit
                    tp = post_order(
                        base=inputs.BASE,
                        quote=inputs.QUOTE,
                        side=side,
                        type='limit',
                        quantity=abs(base_risk), 
                        price=take_px
                    )

                    logging.info(tp)

                    # stop-loss
                    logging.info(stop_px)
                    sl = post_order_stop_loss(
                        base=inputs.BASE, 
                        quote=inputs.QUOTE,
                        side=side,
                        type='stop_loss_limit',
                        quantity=abs(base_risk), 
                        price=stop_px,   
                        stop_price=stop_px
                    )

                    logging.info(sl)

                    global_vars['STOP_LOSS'].append(tp['orderId'])
                    global_vars['STOP_LOSS'].append(sl['orderId'])

                else:
                    logging.debug(f'abs(base_risk): {abs(base_risk)},  mid_px: {round(mid_px, 2)} ,   MAX_USD_RISK: {inputs.MAX_USD_RISK}, cond: {abs(base_risk) * mid_px > inputs.MAX_USD_RISK}')
                    logging.debug('no trading condition met')

        

        #### $$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
        #### $$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
        ### THIS SHOULD SOLVE IMMEDIATE FILLS RISK GOING UNCHECKED
        # Get LIMIT orders
        # Note: OCO orders show up as combo of STOP_LOSS_LIMIT and LIMIT MAKER , and they share same orderIdList
        # opens = get_open_orders(inputs.SYMBOL)
        # open_bid, open_ask = pretty_bid_and_ask(opens)
        # pretty_quote_logging(open_bid, open_ask, key='ending')
        #### $$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
        #### $$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$

        # print avgs
        for k, v in avgs.items():
            global_vars[k] = v
            logging.info(f'{k}: {v}')

        # update global vars
        write_to_pickle(global_vars, inputs.GLOBAL_VAR_PATH)
        logging.info(f'global_vars: {global_vars}')

        # check rate limits
        logging.info(f'rate limit weight for this execution: {get_rate_weight_1m()}')
        logging.info('\n'*3)

        # update heartbeat
        global_vars['HEARTBEAT_TS'] = int(time.time() * 1000)


    start_ws_client(WSClient(callback=main))